# Fine-tune Meta Llama2 13B model provided by Amazon Bedrock: End-to-End

In this notebook we demonstrate using Boto3 sdk for the fine-tuning and provisioning of [Llama2 13B](#https://ai.meta.com/llama/get-started/) model in Bedrock. You can also do this through the Bedrock Console.

<div class="alert alert-block alert-warning">
<b>Warning:</b> This module cannot be executed in Workshop Studio Accounts, and you will have to run this notebook in your own account.
</div>

### A Summarization Use Case
In this notebook, we build an end-to-end workflow for fine-tuning and evaluating the Foundation Models (FMs) in Amazon Bedrock. We choose [Meta Llama 2 13B](https://ai.meta.com/llama/) as our FM to perform the customization through fine-tuning, we then create provisioned throughput of the fine-tuned model, test the provisioned model invocation, and finally evaluate the fine-tuned model performance using [fmeval](https://github.com/aws/fmeval) on the summarization accuracy metrics including METEOR, ROUGE, and BERT scores. We have defined these scores in the `Evaluate the Provisioned Custom Model¶` section below. 

> *This notebook should work well with the **`Data Science 3.0`**, **`Python 3`**, and **`ml.c5.2xlarge`** kernel in SageMaker Studio*

## Prerequisites

 - Make sure you have executed `00_setup.ipynb` notebook.
 - Make sure you are using the same kernel and instance as `00_setup.ipynb` notebook.

In this notebook we demonstrate using Boto3 sdk for the fine-tuning and provisioning of [Llama2 13B](#https://ai.meta.com/llama/get-started/) model in Bedrock. You can also do this through the Bedrock Console.

<div class="alert alert-block alert-warning">
<b>Warning:</b> This notebook will create provisioned throughput for testing the fine-tuned model. Therefore, please make sure to delete the provisioned throughput as mentioned in the last section of the notebook, otherwise you will be charged for it, even if you are not using it.
</div>

## Setup
Install and import all the needed libraries and dependencies to complete this notebook.

Please ignore error messages related to pip's dependency resolver.

In [ ]:
# # install the fmeval package for foundation model evaluation
!rm -Rf ~/.cache/pip/*
!pip install tokenizers==0.12.1
!pip install -qU fmeval==0.3.0

#### Setup Tips:
⚠️ ⚠️ ⚠️ If you have trouble installing fmeval, please make sure you have the dependencies installed correctly. See full list of dependencies [here](https://github.com/aws/fmeval/blob/main/poetry.lock). ⚠️ ⚠️ ⚠️ 


In [ ]:
# restart kernel for packages to take effect
from IPython.core.display import HTML

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
## Fetching varialbes from `00_setup.ipynb` notebook.
%store -r role_arn
%store -r s3_train_uri
%store -r s3_validation_uri
%store -r s3_test_uri
%store -r bucket_name

In [ ]:
import pprint

pprint.pp(role_arn)
pprint.pp(s3_train_uri)
pprint.pp(s3_validation_uri)
pprint.pp(s3_test_uri)
pprint.pp(bucket_name)

In [ ]:
import warnings

warnings.filterwarnings("ignore")
import json
import os
import sys
import boto3
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
session = boto3.session.Session()
region = session.region_name
sts_client = boto3.client("sts")
s3_client = boto3.client("s3")
aws_account_id = sts_client.get_caller_identity()["Account"]
bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [ ]:
test_file_name = "test-cnn-10.jsonl"
data_folder = "fine-tuning-datasets"

## Create the Fine-Tuning Job
<div class="alert alert-block alert-info">
<b>Note:</b> Fine-tuning job will take around 60mins to complete with 5K records.</div>

Meta Llama2 customization hyperparameters: 
- `epochs`: The number of iterations through the entire training dataset and can take up any integer values in the range of 1-10, with a default value of 2.
- `batchSize`: The number of samples processed before updating model parametersand can take up any integer values in the range of 1-64, with a default value of 1.
- `learningRate`:	The rate at which model parameters are updated after each batch	which can take up a float value betweek 0.0-1.0 with a default value set to	1.00E-5.
- `learningRateWarmupSteps`: The number of iterations over which the learning rate is gradually increased to the specified rate and can take any integer value between 0-250 with a default value of 5.

For guidelines on setting hyper-parameters refer to the guidelines provided [here](#https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-guidelines.html)

In [ ]:
from datetime import datetime

ts = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")


# Choose the foundation model you want to customize and provide ModelId(find more about model reference at https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-reference.html)
base_model_id = "meta.llama2-13b-v1:0:4k"

# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING".
customization_type = "FINE_TUNING"

# Specify the roleArn for your customization job
customization_role = role_arn

# Create a customization job name
customization_job_name = f"llama2-finetune-sm-test-model-{ts}"

# Create a customized model name for your fine-tuned Llama2 model
custom_model_name = f"llama2-finetune-{ts}"

# Define the hyperparameters for fine-tuning Llama2 model
hyper_parameters = {
    "epochCount": "2",
    "batchSize": "1",
    "learningRate": "0.00005",
}

# Specify your data path for training, validation(optional) and output
training_data_config = {"s3Uri": s3_train_uri}

# # uncomment the below section if you have validation dataset and provide the s3 uri for it.
validation_data_config = {"validators": [{"s3Uri": s3_validation_uri}]}

output_data_config = {"s3Uri": f"s3://{bucket_name}/outputs/output-{custom_model_name}"}

# # Create the customization job
bedrock.create_model_customization_job(
    customizationType=customization_type,
    jobName=customization_job_name,
    customModelName=custom_model_name,
    roleArn=customization_role,
    baseModelIdentifier=base_model_id,
    hyperParameters=hyper_parameters,
    trainingDataConfig=training_data_config,
    validationDataConfig=validation_data_config,
    outputDataConfig=output_data_config,
)

## Check Customization Job Status

In [ ]:
import time

fine_tune_job = bedrock.get_model_customization_job(
    jobIdentifier=customization_job_name
)["status"]
print(fine_tune_job)

while fine_tune_job == "InProgress":
    time.sleep(60)
    fine_tune_job = bedrock.get_model_customization_job(
        jobIdentifier=customization_job_name
    )["status"]
    print(fine_tune_job)

## Retrieve Custom Model
Once the customization job is finished, you can check your existing custom model(s) and retrieve the modelArn of your fine-tuned Llama2 model.

In [ ]:
# You can list your custom models using the command below
bedrock.list_custom_models()

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure your customization job status is "completed" before proceeding to retrieve the modelArn, otherwise you will run into errors. </div>


In [ ]:
# retrieve the modelArn of the fine-tuned model
fine_tune_job = bedrock.get_custom_model(modelIdentifier=custom_model_name)
custom_model_id = fine_tune_job["modelArn"]

In [ ]:
output_job_name = "model-customization-job-" + fine_tune_job["jobArn"].split("/")[-1]
output_job_name

## Visualize Training and Validation Loss
Now that we have completed fine-tuning job, lets visualize our results to see if our job is not underfitting or overfitting. 

Download model customization job metrics from S3 and plot the learning curves.

In [ ]:
output_metrics_path = f"fine-tuning-datasets/{output_job_name}"

In [ ]:
!mkdir $output_metrics_path

In [ ]:
train_metrics_s3_prefix = f"outputs/output-{custom_model_name}/{output_job_name}/training_artifacts/step_wise_training_metrics.csv"
validation_metrics_s3_prefix = f"outputs/output-{custom_model_name}/{output_job_name}/validation_artifacts/post_fine_tuning_validation/validation/validation_metrics.csv"
train_metrics_name = "train_metrics.csv"
validation_metrics_name = "validation_metrics.csv"
train_file_name_local = output_metrics_path + "/" + train_metrics_name
validation_file_name_local = output_metrics_path + "/" + validation_metrics_name

In [ ]:
s3_client.download_file(bucket_name, train_metrics_s3_prefix, train_file_name_local)
s3_client.download_file(
    bucket_name, validation_metrics_s3_prefix, validation_file_name_local
)

In [ ]:
train_data = pd.read_csv(train_file_name_local)
"""The training loss is at an iteration level. To calculate loss at the epoch level,
    average the iteration-level loss for each epoch"""
train_metrics_epoch = train_data.groupby("epoch_number").mean()
validation_metrics_epoch = pd.read_csv(validation_file_name_local)

In [ ]:
plt.plot(
    validation_metrics_epoch.epoch_number,
    validation_metrics_epoch.validation_loss,
    label="validation",
)
plt.plot(train_metrics_epoch.index, train_metrics_epoch.training_loss, label="training")
plt.title("Training vs Validation Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()

## Create Provisioned Throughput
<div class="alert alert-block alert-info">
<b>Note:</b> Creating provisioned throughput will take around 20-30mins to complete.</div>
You will need to create provisioned throughput to be able to evaluate the model performance. You can do so through the [console](https://docs.aws.amazon.com/bedrock/latest/userguide/prov-cap-console.html) or use the following api call.

In [ ]:
# Create the provision throughput job and retrieve the provisioned model id
provisioned_model_id = bedrock.create_provisioned_model_throughput(
    modelUnits=1,
    # create a name for your provisioned throughput model
    provisionedModelName="test-model-v1-001",
    modelId=custom_model_id,
)["provisionedModelArn"]

In [ ]:
# check provisioned throughput job status
import time

status_provisioning = bedrock.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_id
)["status"]
while status_provisioning == "Creating":
    time.sleep(60)
    status_provisioning = bedrock.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_id
    )["status"]
    print(status_provisioning)

## Invoke the Provisioned Custom Model
Invoke the privisioned custom model.You can replace the follwing prompt_txt with the prompts that are more similar to your fine-tuning dataset, this helps to check whether the fine-tuned model is performing as you expected. 

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure your provisioned throughput job status becomes InService before proceeding. </div>

In [ ]:
# Provide the prompt text
test_file_path = f"{data_folder}/{test_file_name}"
with open(test_file_path) as f:
    lines = f.read().splitlines()

In [ ]:
test_prompt = json.loads(lines[0])["prompt"]
reference_summary = json.loads(lines[0])["completion"]
print(test_prompt)
print()
print(reference_summary)

Construct model input following the format needed by Llama2 model following instructions [here](#https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-meta.html).
Please pay attention to the "Model invocation request body field" section

In [ ]:
body = json.dumps(
    {
        "prompt": test_prompt,
        # specify the parameters as needed
        "max_gen_len": 200,
        "temperature": 0.4,
        "top_p": 0.3,
    }
)

# provide the modelId of the provisioned custom model
modelId = provisioned_model_id
accept = "application/json"
contentType = "application/json"

# invoke the provisioned custom model
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)

response_body = json.loads(response.get("body").read())
print(response_body)

## Clean up
<div class="alert alert-block alert-warning">
<b>Warning:</b> Please make sure to delete providsioned throughput with the following code as there will be cost incurred if its left in running state, even if you are not using it. 
</div>

In [ ]:
# delete the provisioned throughput
bedrock.delete_provisioned_model_throughput(provisionedModelId=provisioned_model_id)

<div class="alert alert-block alert-info">
<b>Note:</b> Please finish up the cleaning process by running 04_cleanup.ipynb to clean up the other resources. </div>